In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

In [3]:
train = pd.read_csv('C:/Users/user/LikeLion_AI/data/CSV/DACON_Parking_Demand/train.csv')
test = pd.read_csv('C:/Users/user/LikeLion_AI/data/CSV/DACON_Parking_Demand/test.csv')
sub = pd.read_csv('C:/Users/user/LikeLion_AI/data/CSV/DACON_Parking_Demand/sample_submission.csv')
age = pd.read_csv('C:/Users/user/LikeLion_AI/data/CSV/DACON_Parking_Demand/age_gender_info.csv')

train.shape, test.shape, sub.shape, age.shape

((2952, 15), (1022, 14), (150, 2), (16, 23))

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2952 entries, 0 to 2951
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          2952 non-null   object 
 1   총세대수                          2952 non-null   int64  
 2   임대건물구분                        2952 non-null   object 
 3   지역                            2952 non-null   object 
 4   공급유형                          2952 non-null   object 
 5   전용면적                          2952 non-null   float64
 6   전용면적별세대수                      2952 non-null   int64  
 7   공가수                           2952 non-null   float64
 8   자격유형                          2952 non-null   object 
 9   임대보증금                         2383 non-null   object 
 10  임대료                           2383 non-null   object 
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  2741 non-null   float64
 12  도보 10분거리 내 버스정류장 수            2948 non-null   float64
 13  단지내

In [5]:
train.isnull().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              0
임대보증금                           569
임대료                             569
도보 10분거리 내 지하철역 수(환승노선 수 반영)    211
도보 10분거리 내 버스정류장 수                4
단지내주차면수                           0
등록차량수                             0
dtype: int64

train1 : 전용 면적 당 임대보증금과 임대료를 예측하기 위한 DataFrame

#### 그리고 오류가 있는 단지코드를 모두 삭제

In [6]:
train1 = train.copy()

drop_danji = ['C2085', 'C1397', 'C2431', 'C1649', 'C1036', 
              'C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988']    # 총 12개


print(len(train1['단지코드'].unique()))   # drop 이전 : 423
for code in drop_danji:
    train1 = train1[train1['단지코드'] != code]

print(len(train1['단지코드'].unique()))   # drop 이후 : 411
print()

print((train1[train1['임대건물구분']=='아파트']['임대료'] == '-').sum())
print(train1[train1['임대건물구분']=='아파트']['임대료'].isna().sum())

print((train1[train1['임대건물구분']=='상가']['임대료'] == '-').sum())
print(train1[train1['임대건물구분']=='상가']['임대료'].isna().sum())

423
411

15
7
0
562


In [7]:
train1['임대보증금/전용면적'] = 0
train1['임대료/전용면적'] = 0

train1.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수,임대보증금/전용면적,임대료/전용면적
8,C2515,545,아파트,경상남도,국민임대,33.48,276,17.0,A,9216000,82940,0.0,3.0,624.0,205.0,0,0
9,C2515,545,아파트,경상남도,국민임대,39.60,60,17.0,A,12672000,107130,0.0,3.0,624.0,205.0,0,0
10,C2515,545,아파트,경상남도,국민임대,39.60,20,17.0,A,12672000,107130,0.0,3.0,624.0,205.0,0,0
11,C2515,545,아파트,경상남도,국민임대,46.90,38,17.0,A,18433000,149760,0.0,3.0,624.0,205.0,0,0
12,C2515,545,아파트,경상남도,국민임대,46.90,19,17.0,A,18433000,149760,0.0,3.0,624.0,205.0,0,0


### 단지 별 임대료 결측치 

In [8]:
danji = train['단지코드'].value_counts().index
danji_list = []

for code in danji:
    null_sum = train1[train1['단지코드']==code]['임대료'].isnull().sum() 
    null_sum += (train1[train1['단지코드']==code]['임대료'] == '-').sum()
    if null_sum:
        danji_list.append(code)
        print(f"{code}의 갯수 = {len(train1[train1['단지코드']==code])}")
        print(f'{code}의 임대료 결측치 갯수 = {null_sum}')
        print(train1[train1['단지코드']==code]['임대건물구분'].value_counts())
        print()

C1439의 갯수 = 48
C1439의 임대료 결측치 갯수 = 45
상가     45
아파트     3
Name: 임대건물구분, dtype: int64

C2132의 갯수 = 47
C2132의 임대료 결측치 갯수 = 43
상가     43
아파트     4
Name: 임대건물구분, dtype: int64

C1859의 갯수 = 40
C1859의 임대료 결측치 갯수 = 35
상가     35
아파트     5
Name: 임대건물구분, dtype: int64

C1790의 갯수 = 33
C1790의 임대료 결측치 갯수 = 30
상가     30
아파트     3
Name: 임대건물구분, dtype: int64

C2597의 갯수 = 32
C2597의 임대료 결측치 갯수 = 29
상가     29
아파트     3
Name: 임대건물구분, dtype: int64

C1874의 갯수 = 30
C1874의 임대료 결측치 갯수 = 27
상가     27
아파트     3
Name: 임대건물구분, dtype: int64

C2034의 갯수 = 29
C2034의 임대료 결측치 갯수 = 26
상가     26
아파트     3
Name: 임대건물구분, dtype: int64

C2289의 갯수 = 24
C2289의 임대료 결측치 갯수 = 20
상가     20
아파트     4
Name: 임대건물구분, dtype: int64

C2310의 갯수 = 23
C2310의 임대료 결측치 갯수 = 20
상가     20
아파트     3
Name: 임대건물구분, dtype: int64

C1616의 갯수 = 22
C1616의 임대료 결측치 갯수 = 19
상가     19
아파트     3
Name: 임대건물구분, dtype: int64

C1109의 갯수 = 22
C1109의 임대료 결측치 갯수 = 20
상가     20
아파트     2
Name: 임대건물구분, dtype: int64

C1875의 갯수 = 22
C1875의 임대료 결측치 갯수 = 19
상가     19
아파트   

#### 결측치가 전체의 과반수를 차지하고 있는 상황이 대부분.. 어떻게 처리해야할까?

1. 우선 다 삭제하고 해본다.

2. 과반수 이하가 있는 곳만 평균값으로 살린다.

3. 전용면적을 비율로 임대료를 각각 계산한다.

4. 시세를 예측해서 진행해본다.

#### -> 뭔가 3번이 제일 끌리니까 3번으로 바로 해보자!

15평 -> 500/25(기준) 

100평 -> 3000/120(번화가) >> 임대아파트

400평 -> 10000/400(예상)

In [9]:
train1.loc[train1['단지코드']==danji_list[2], :]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수,임대보증금/전용면적,임대료/전용면적
261,C1859,2424,아파트,대전광역시,영구임대,26.37,1865,1.0,C,5787000,79980,1.0,4.0,487.0,116.0,0,0
262,C1859,2424,아파트,대전광역시,영구임대,26.37,10,1.0,C,5787000,79980,1.0,4.0,487.0,116.0,0,0
263,C1859,2424,아파트,대전광역시,영구임대,26.37,10,1.0,C,5787000,79980,1.0,4.0,487.0,116.0,0,0
264,C1859,2424,아파트,대전광역시,영구임대,31.32,478,1.0,C,6873000,94990,1.0,4.0,487.0,116.0,0,0
265,C1859,2424,아파트,대전광역시,영구임대,52.74,26,1.0,C,11574000,159960,1.0,4.0,487.0,116.0,0,0
266,C1859,2424,상가,대전광역시,임대상가,38.00,1,1.0,D,NaN,NaN,1.0,4.0,487.0,116.0,0,0
267,C1859,2424,상가,대전광역시,임대상가,38.00,1,1.0,D,NaN,NaN,1.0,4.0,487.0,116.0,0,0
268,C1859,2424,상가,대전광역시,임대상가,31.84,1,1.0,D,NaN,NaN,1.0,4.0,487.0,116.0,0,0
269,C1859,2424,상가,대전광역시,임대상가,31.84,1,1.0,D,NaN,NaN,1.0,4.0,487.0,116.0,0,0
270,C1859,2424,상가,대전광역시,임대상가,31.84,1,1.0,D,NaN,NaN,1.0,4.0,487.0,116.0,0,0


#### 단지 별로 임대건물구분, 공급유형, 전용면적, 전용면적별세대수, 자격유형, 임대보증급, 임대료 빼고 다 똑같구나..

#### 공가수 : 데이터 수집 당시 임차되고 있지 않은 빈 집

In [10]:
## 임대보증금과 임대료의 NaN 값과 - 값을 0으로 변환
train1.loc[train1['임대보증금'].isna(), '임대보증금'] = '0'
train1.loc[train1['임대료'].isna(), '임대료'] = '0'

train1.loc[train1['임대보증금']=='-', '임대보증금'] = '0'
train1.loc[train1['임대료']=='-', '임대료'] = '0'

train1['임대보증금'] = train1['임대보증금'].astype('float')
train1['임대료'] = train1['임대료'].astype('float')
train1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2869 entries, 8 to 2951
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          2869 non-null   object 
 1   총세대수                          2869 non-null   int64  
 2   임대건물구분                        2869 non-null   object 
 3   지역                            2869 non-null   object 
 4   공급유형                          2869 non-null   object 
 5   전용면적                          2869 non-null   float64
 6   전용면적별세대수                      2869 non-null   int64  
 7   공가수                           2869 non-null   float64
 8   자격유형                          2869 non-null   object 
 9   임대보증금                         2869 non-null   float64
 10  임대료                           2869 non-null   float64
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  2662 non-null   float64
 12  도보 10분거리 내 버스정류장 수            2869 non-null   float64
 13  단지내

In [11]:
train1 = train1.reset_index()

In [12]:
## 임대료에 결측치가 포함된 단지코드와 단위 면적당 임대료 및 임대보증금의 평균을 rent_dict에 저장
rent_dict = {}

for code in danji_list:
    temp_data = train1.loc[train1['단지코드']==code, :]
    for i in temp_data.index:
        if temp_data.loc[i, '임대료']:   # 임대료가 NaN이 아닌 경우
            deposit_list = []
            rental_list = []
            
            area = temp_data.loc[i, '전용면적']
            deposit = int(temp_data.loc[i, '임대보증금'])
            rental = int(temp_data.loc[i, '임대료'])
            
            deposit_list.append(1/area*deposit)
            rental_list.append(1/area*rental)
            
            deposit_mean = round(np.mean(deposit_list), 3)
            rental_mean = round(np.mean(rental_list), 3)
            
            rent_dict[code] = [deposit_mean, rental_mean]

```python
for code, value in rent_dict.items():
    temp_data = train1.loc[train1['단지코드']==code, :]
    for i in temp_data.index:
        if not temp_data.loc[i, '임대료']:   # 임대료가 빈 경우
            temp_data['임대보증금'] = value[0] * temp_data.loc[i, '전용면적']
            temp_data['임대료'] = value[1] * temp_data.loc[i, '전용면적']
```

DataFrame 접근방식이 틀려서 이렇게 변경이 안되는듯

In [14]:
## 결측치가 포함된 단지코드의 단위면적당 임대료/임대보증금과 전용면적을 곱한 값을 기존의 임대보증금과 임대료의 수치에 덮어씌움
for code, value in rent_dict.items():
    train1.loc[train1['단지코드']==code, '임대보증금/전용면적'] = value[0]
    train1.loc[train1['단지코드']==code, '임대료/전용면적'] = value[1]
    
    train1.loc[train1['단지코드']==code, '임대보증금'] = value[0] * train1.loc[train1['단지코드']==code]['전용면적']
    train1.loc[train1['단지코드']==code, '임대료'] = value[1] * train1.loc[train1['단지코드']==code]['전용면적']

In [26]:
## 상가는 아파트와 시세가 다르기 때문에 같은 면적 대비 임대보증금을 1.2배 해주고 임대료를 10배 해준다.
for code in rent_dict.keys():
    train1.loc[(train1['단지코드']==code) & (train1['임대건물구분']=='상가'), '임대보증금']\
    = train1.loc[(train1['단지코드']==code) & (train1['임대건물구분']=='상가')]['임대보증금'] * 1.2
    
    train1.loc[(train1['단지코드']==code) & (train1['임대건물구분']=='상가'), '임대료']\
    = train1.loc[(train1['단지코드']==code) & (train1['임대건물구분']=='상가')]['임대료'] * 10

In [20]:
## 결측치 확인
print((train1[train1['임대건물구분']=='아파트']['임대료'] == '-').sum())
print(train1[train1['임대건물구분']=='아파트']['임대료'].isna().sum())

print((train1[train1['임대건물구분']=='상가']['임대료'] == '-').sum())
print(train1[train1['임대건물구분']=='상가']['임대료'].isna().sum())

0
0
0
0


#### 지하철 역 결측치가 있는 단지코드 출력

In [40]:
train1[train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].isnull()]['단지코드'].value_counts()

C1874    30
C1875    22
C1616    22
C2258    20
C1004    19
C1704    17
C1983    15
C1350    13
C2583    11
C2644     7
C2100     6
C1068     6
C1312     4
C1175     3
C1005     3
C1424     3
C2156     3
C2216     2
C2520     1
Name: 단지코드, dtype: int64

#### 위의 단지코드에는 지하철 역 값이 전혀 없음.

1. 아예 삭제한다.

2. 최빈값인 0을 대입한다.

3. 결측치가 있는 각 단지의 등록차량수를 평균내어 지하철 역 값에 따른 등록차량수의 평균과 비교해 제일 비슷한 값으로 대입한다.

In [54]:
train1.groupby('도보 10분거리 내 지하철역 수(환승노선 수 반영)')['등록차량수'].mean(), train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].value_counts()

(도보 10분거리 내 지하철역 수(환승노선 수 반영)
 0.0    587.101664
 1.0    441.430000
 2.0    689.363636
 3.0    166.000000
 Name: 등록차량수, dtype: float64,
 0.0    2223
 1.0     400
 2.0      33
 3.0       6
 Name: 도보 10분거리 내 지하철역 수(환승노선 수 반영), dtype: int64)

In [49]:
## 지하철역의 결측치가 존재하는 단지코드의 등록차량수 출력
train1[train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].isnull()]\
.drop_duplicates(['단지코드'], keep='first')[['단지코드', '등록차량수']]

,단지코드,등록차량수
86,C1312,359.0
90,C1874,62.0
136,C1424,179.0
139,C2100,671.0
163,C2520,386.0
168,C1616,98.0
190,C1704,131.0
207,C2258,78.0
339,C1068,510.0
345,C1983,253.0


#### 지하철 역 수에 대해 전체적으로 정보를 출력

In [61]:
print(train1[train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)']==0]['등록차량수'].describe())
print(train1[train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)']==0]['단지코드'].value_counts())
print(len(train1[train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)']==0]['단지코드'].value_counts()))
print()

print(train1[train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)']==1]['등록차량수'].describe())
print(train1[train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)']==1]['단지코드'].value_counts())
print(len(train1[train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)']==1]['단지코드'].value_counts()))
print()

print(train1[train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)']==2]['등록차량수'].describe())
print(train1[train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)']==2]['단지코드'].value_counts())
print(len(train1[train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)']==2]['단지코드'].value_counts()))
print()

print(train1[train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)']==3]['등록차량수'].describe())
print(train1[train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)']==3]['단지코드'].value_counts())
print(len(train1[train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)']==3]['단지코드'].value_counts()))
print()

count    2223.000000
mean      587.101664
std       417.936131
min        16.000000
25%       293.000000
50%       509.000000
75%       787.000000
max      2550.000000
Name: 등록차량수, dtype: float64
C2132    47
C1790    33
C2597    32
C2289    24
C2310    23
         ..
C1487     2
C1788     2
C2188     1
C2340     1
C2437     1
Name: 단지코드, Length: 348, dtype: int64
348

count     400.000000
mean      441.430000
std       389.544551
min        13.000000
25%       132.000000
50%       313.000000
75%       649.000000
max      1668.000000
Name: 등록차량수, dtype: float64
C1439    48
C1859    40
C2034    29
C1109    22
C2135    20
         ..
C1155     3
C2020     3
C2470     3
C1613     3
C1497     2
Name: 단지코드, Length: 38, dtype: int64
38

count      33.000000
mean      689.363636
std       218.308672
min       508.000000
25%       548.000000
50%       559.000000
75%       757.000000
max      1077.000000
Name: 등록차량수, dtype: float64
C2008    10
C1122     7
C2453     7
C2042     5
C1757     4
Name

-> 아무래도 표본이 0과 1에 치중이 되어있다보니 등록차량수는 정확한 지표로 사용하기 힘들 것 같다.

-> 일반적으로 지하철 역이 주변에 3개나 있으면 차량을 많이 가지고 있지 않을 것이다.

or 역이 주변에 많기 때문에 집값도 비쌀 것이고 그만큼 차도 많을 것이다.

#### 그냥 0으로 몰아넣자